In [20]:
import numpy as np
def print_shape(array):
    result =""
    for dim in array:
        result += str(dim) + "x"
    result += "i32"
    return result

def print_data(array):
    return np.array2string(array.astype(np.int32), separator=', ', formatter={'int_kind': lambda x: str(x)}, max_line_width=np.inf, threshold=array.size).replace("\n", "")



def write_file(const_size):
    data = np.zeros(const_size)
    shape = print_shape(data.shape)
    values = print_data(data)
    template =  f"""
            module {{
            func.func @main() {{
                "engine.settings"() {{ value = 1 : i32, type = 0 : i32 }} : () -> ()
                %_model.fc1.weight = "engine.constant"() {{value=dense<{values}>:tensor<{shape}>}} : () -> memref<{shape}>

                func.return 
            }}
            }}
        """

    with open('stress.mlir', 'w+') as f:
        f.write(template)
    return template

In [27]:
import subprocess
import time
from tqdm import tqdm
script_path = "./hlstool_test.sh" 

execs = []
end =100
for i in tqdm(range(0, end), desc="Running tests", unit="iter"):
    write_file((i+1))
    start_time = time.time()
    result = subprocess.run([script_path], capture_output=True, text=True)
    end_time = time.time()
    execs.append(end_time - start_time)

import matplotlib.pyplot as plt

plt.plot(range(0,end), execs)
plt.ylabel("Execution Time (s)")
plt.xlabel("Constant Size")
plt.show()

Running tests:  69%|██████▉   | 69/100 [07:33<08:36, 16.66s/iter]